# TASK 7

CODE 47 -  Count Inversions

In [1]:
def count_inversions(arr):
    # Base case: if the array has 0 or 1 element, there are no inversions
    if len(arr) <= 1:
        return arr, 0
    
    # Split the array into two halves
    mid = len(arr) // 2
    left, inv_left = count_inversions(arr[:mid])
    right, inv_right = count_inversions(arr[mid:])
    
    # Merge the two halves and count split inversions
    merged, inv_merge = merge_and_count(left, right)
    
    # Total inversions = left inversions + right inversions + split inversions
    total_inv = inv_left + inv_right + inv_merge
    return merged, total_inv

def merge_and_count(left, right):
    result = []
    inversions = 0
    i = j = 0
    
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
            # All remaining elements in left[i:] are greater than right[j]
            inversions += len(left) - i
    
    # Add any remaining elements
    result.extend(left[i:])
    result.extend(right[j:])
    
    return result, inversions


if __name__ == "__main__":
    # Test cases
    test_cases = [
        ([1, 3, 5, 2, 4, 6], 3),
        ([5, 4, 3, 2, 1], 10),
        ([1, 2, 3, 4, 5], 0),
        ([2, 4, 1, 3, 5], 3),
        ([], 0),
        ([1], 0)
    ]
    
    for arr, expected in test_cases:
        _, count = count_inversions(arr)
        print(f"Array: {arr}")
        print(f"Expected inversions: {expected}")
        print(f"Computed inversions: {count}")
        print(f"{'PASS' if count == expected else 'FAIL'}")
        print("-" * 40)

Array: [1, 3, 5, 2, 4, 6]
Expected inversions: 3
Computed inversions: 3
PASS
----------------------------------------
Array: [5, 4, 3, 2, 1]
Expected inversions: 10
Computed inversions: 10
PASS
----------------------------------------
Array: [1, 2, 3, 4, 5]
Expected inversions: 0
Computed inversions: 0
PASS
----------------------------------------
Array: [2, 4, 1, 3, 5]
Expected inversions: 3
Computed inversions: 3
PASS
----------------------------------------
Array: []
Expected inversions: 0
Computed inversions: 0
PASS
----------------------------------------
Array: [1]
Expected inversions: 0
Computed inversions: 0
PASS
----------------------------------------


CODE 48 - Find the Longest Palindromic Substring

In [2]:
def longest_palindromic_substring(s: str) -> str:
    if not s:
        return ""
    
    start = 0
    end = 0
    
    for i in range(len(s)):
        # Check for odd-length palindromes (centered at i)
        len1 = expand_around_center(s, i, i)
        # Check for even-length palindromes (centered between i and i+1)
        len2 = expand_around_center(s, i, i + 1)
        # Take the maximum length
        max_len = max(len1, len2)
        
        # Update start and end indices if a longer palindrome is found
        if max_len > end - start:
            start = i - (max_len - 1) // 2
            end = i + max_len // 2
    
    return s[start:end + 1]

def expand_around_center(s: str, left: int, right: int) -> int:
    # Expand as long as characters match and within bounds
    while left >= 0 and right < len(s) and s[left] == s[right]:
        left -= 1
        right += 1
    # Return the length of the palindrome found
    return right - left - 1


if __name__ == "__main__":
    test_cases = [
        ("babad", ["bab", "aba"]),
        ("cbbd", ["bb"]),
        ("a", ["a"]),
        ("ac", ["a", "c"]),
        ("racecar", ["racecar"]),
        ("", [""]),
    ]
    
    for s, expected in test_cases:
        result = longest_palindromic_substring(s)
        print(f"Input: '{s}'")
        print(f"Expected: {expected}")
        print(f"Result: '{result}'")
        print(f"{'PASS' if result in expected else 'FAIL'}")
        print("-" * 40)

Input: 'babad'
Expected: ['bab', 'aba']
Result: 'aba'
PASS
----------------------------------------
Input: 'cbbd'
Expected: ['bb']
Result: 'bb'
PASS
----------------------------------------
Input: 'a'
Expected: ['a']
Result: 'a'
PASS
----------------------------------------
Input: 'ac'
Expected: ['a', 'c']
Result: 'c'
PASS
----------------------------------------
Input: 'racecar'
Expected: ['racecar']
Result: 'racecar'
PASS
----------------------------------------
Input: ''
Expected: ['']
Result: ''
PASS
----------------------------------------


CODE 49 - Traveling Salesman Problem (TSP)

In [5]:
import sys
from itertools import combinations

def tsp_dp(distances):
    n = len(distances)
    if n == 0:
        return [], 0
    if n == 1:
        return [0], 0
    
    # Memoization table: key=(subset, last_node), value=(min_cost, prev_node)
    memo = {}
    
    # Initialize for subsets of size 1
    for k in range(1, n):
        memo[(frozenset([k]), k)] = (distances[0][k], 0)
    
    # Iterate over subset sizes
    for s in range(2, n):
        for subset in combinations(range(1, n), s):
            subset = frozenset(subset)
            for k in subset:
                # Minimum cost to reach k from 0 via subset-{k}
                min_cost = sys.maxsize
                prev_node = None
                for m in subset:
                    if m == k:
                        continue
                    cost = memo[(subset - {k}, m)][0] + distances[m][k]
                    if cost < min_cost:
                        min_cost = cost
                        prev_node = m
                memo[(subset, k)] = (min_cost, prev_node)
    
    # Find the minimum cost to complete the tour
    full_set = frozenset(range(1, n))
    min_cost = sys.maxsize
    last_node = None
    for k in range(1, n):
        cost = memo[(full_set, k)][0] + distances[k][0]
        if cost < min_cost:
            min_cost = cost
            last_node = k
    
    # Reconstruct the path
    path = [0]
    subset = full_set
    node = last_node
    while subset:
        path.append(node)
        prev_node = memo[(subset, node)][1]
        subset = subset - {node}
        node = prev_node
    path.append(0)
    
    return path, min_cost

# Example usage:
if __name__ == "__main__":
    # Example distance matrix (0 is the starting city)
    dist = [
        [0, 10, 15, 20],
        [10, 0, 35, 25],
        [15, 35, 0, 30],
        [20, 25, 30, 0]
    ]
    
    path, cost = tsp_dp(dist)
    print("Optimal path:", path)
    print("Total distance:", cost)

Optimal path: [0, 1, 3, 2, 0]
Total distance: 80


CODE 50 - Graph Cycle Detection

In [4]:
def is_cyclic_directed(graph):
    visited = set()
    recursion_stack = set()
    
    def dfs(node):
        visited.add(node)
        recursion_stack.add(node)
        for neighbor in graph.get(node, []):
            if neighbor not in visited:
                if dfs(neighbor):
                    return True
            elif neighbor in recursion_stack:
                return True
        recursion_stack.remove(node)
        return False
    
    for node in graph:
        if node not in visited:
            if dfs(node):
                return True
    return False

# Example usage:
directed_graph = {
    0: [1],
    1: [2],
    2: [0]
}

print(is_cyclic_directed(directed_graph))  # Output: True

True


CODE 51 - Longest Substring Without Repeating Characters

In [6]:
def length_of_longest_substring(s: str) -> int:
    char_index = {}  # Stores the most recent index of each character
    start = max_length = 0
    
    for end, char in enumerate(s):
        # If we've seen the character before, update the start position
        if char in char_index:
            start = max(start, char_index[char] + 1)
        
        # Update the character's last seen index
        char_index[char] = end
        
        # Update the maximum length if needed
        max_length = max(max_length, end - start + 1)
    
    return max_length

# Example usage:
if __name__ == "__main__":
    test_cases = [
        ("abcabcbb", 3),
        ("bbbbb", 1),
        ("pwwkew", 3),
        ("", 0),
        (" ", 1),
        ("dvdf", 3)
    ]
    
    for s, expected in test_cases:
        result = length_of_longest_substring(s)
        print(f"Input: '{s}'")
        print(f"Expected: {expected}")
        print(f"Result: {result}")
        print(f"{'PASS' if result == expected else 'FAIL'}")
        print("-" * 40)

Input: 'abcabcbb'
Expected: 3
Result: 3
PASS
----------------------------------------
Input: 'bbbbb'
Expected: 1
Result: 1
PASS
----------------------------------------
Input: 'pwwkew'
Expected: 3
Result: 3
PASS
----------------------------------------
Input: ''
Expected: 0
Result: 0
PASS
----------------------------------------
Input: ' '
Expected: 1
Result: 1
PASS
----------------------------------------
Input: 'dvdf'
Expected: 3
Result: 3
PASS
----------------------------------------


CODE 52 - Find All Valid Parentheses Combinations

In [7]:
def generateParenthesis(n):
    def backtrack(current, open_count, close_count, result):
        if len(current) == 2 * n:
            result.append(current)
            return
        if open_count < n:
            backtrack(current + '(', open_count + 1, close_count, result)
        if close_count < open_count:
            backtrack(current + ')', open_count, close_count + 1, result)
    
    result = []
    backtrack("", 0, 0, result)
    return result

# Example usage:
if __name__ == "__main__":
    test_cases = [1, 2, 3]
    for n in test_cases:
        print(f"n = {n}:")
        print(generateParenthesis(n))
        print("-" * 30)

n = 1:
['()']
------------------------------
n = 2:
['(())', '()()']
------------------------------
n = 3:
['((()))', '(()())', '(())()', '()(())', '()()()']
------------------------------


CODE 53 - Zigzag Level Order Traversal of Binary Tree

In [8]:
from collections import deque

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def zigzagLevelOrder(root):
    if not root:
        return []
    
    result = []
    current_level = deque([root])
    left_to_right = True
    
    while current_level:
        level_size = len(current_level)
        current_values = []
        next_level = deque()
        
        for _ in range(level_size):
            if left_to_right:
                node = current_level.popleft()
                current_values.append(node.val)
                if node.left:
                    next_level.append(node.left)
                if node.right:
                    next_level.append(node.right)
            else:
                node = current_level.pop()
                current_values.append(node.val)
                if node.right:
                    next_level.appendleft(node.right)
                if node.left:
                    next_level.appendleft(node.left)
        
        result.append(current_values)
        current_level = next_level
        left_to_right = not left_to_right
    
    return result

# Example usage:
if __name__ == "__main__":
    # Example tree:
    #     3
    #    / \
    #   9  20
    #     /  \
    #    15   7
    root = TreeNode(3)
    root.left = TreeNode(9)
    root.right = TreeNode(20)
    root.right.left = TreeNode(15)
    root.right.right = TreeNode(7)
    
    print(zigzagLevelOrder(root))  # Output: [[3], [20, 9], [15, 7]]

[[3], [20, 9], [15, 7]]


CODE 54 - Palindrome Partitioning

In [9]:
def partition(s: str):
    def is_palindrome(sub):
        return sub == sub[::-1]
    
    def backtrack(start, path):
        if start == len(s):
            result.append(path.copy())
            return
        for end in range(start + 1, len(s) + 1):
            substring = s[start:end]
            if is_palindrome(substring):
                path.append(substring)
                backtrack(end, path)
                path.pop()
    
    result = []
    backtrack(0, [])
    return result

# Example usage:
if __name__ == "__main__":
    test_cases = ["aab", "racecar", "a", ""]
    for s in test_cases:
        print(f"Input: '{s}'")
        print("Partitions:", partition(s))
        print("-" * 40)

Input: 'aab'
Partitions: [['a', 'a', 'b'], ['aa', 'b']]
----------------------------------------
Input: 'racecar'
Partitions: [['r', 'a', 'c', 'e', 'c', 'a', 'r'], ['r', 'a', 'cec', 'a', 'r'], ['r', 'aceca', 'r'], ['racecar']]
----------------------------------------
Input: 'a'
Partitions: [['a']]
----------------------------------------
Input: ''
Partitions: [[]]
----------------------------------------


# Personal Budget Advisor

In [ ]:
# Personal Budget Advisor -

transactions = []

# Load existing data
def load_data(filename="budget_data.txt"):
    global transactions
    try:
        with open(filename, "r") as file:
            transactions = [eval(line.strip()) for line in file.readlines()]
        print("Data loaded successfully.")
    except FileNotFoundError:
        print("No existing data found. Starting fresh.")

# Save data to file
def save_data(filename="budget_data.txt"):
    with open(filename, "w") as file:
        for transaction in transactions:
            file.write(str(transaction) + "\n")
    print("Data saved successfully.")

# Add a transaction
def add_transaction():
    try:
        t_type = input("Enter type (income/expense): ").strip().lower()
        if t_type not in ["income", "expense"]:
            raise ValueError("Type must be 'income' or 'expense'")
        amount = float(input("Enter amount: "))
        if amount <= 0:
            raise ValueError("Amount must be positive.")
        category = input("Enter category (e.g., food, rent, salary): ").strip().lower()
        description = input("Enter description: ").strip()
        transactions.append({
            "type": t_type,
            "amount": amount,
            "category": category,
            "description": description
        })
        print("Transaction added successfully!")
    except ValueError as ve:
        print(f"Error: {ve}")

# View income/expense summary
def view_summary():
    income_total = sum(t["amount"] for t in transactions if t["type"] == "income")
    expense_total = sum(t["amount"] for t in transactions if t["type"] == "expense")
    balance = income_total - expense_total
    print("\n--- Summary ---")
    print(f"Total Income: ₹{income_total:.2f}")
    print(f"Total Expenses: ₹{expense_total:.2f}")
    print(f"Balance: ₹{balance:.2f}")
    if income_total > 0:
        expense_percent = (expense_total / income_total) * 100
        print(f"Percentage of Income Spent: {expense_percent:.2f}%")
    else:
        print("No income recorded yet.")

# Analyze spending by category
def analyze_spending():
    category_totals = {}
    for t in transactions:
        if t["type"] == "expense":
            category_totals[t["category"]] = category_totals.get(t["category"], 0) + t["amount"]
    if not category_totals:
        print("No expenses to analyze.")
        return
    print("\n--- Spending by Category ---")
    for cat, amt in category_totals.items():
        print(f"{cat.capitalize()}: ₹{amt:.2f}")
    top_category = max(category_totals, key=category_totals.get)
    print(f"\nYou spend the most on: {top_category.capitalize()}")

# Rule-based budget suggestion
def suggest_savings():
    income_total = sum(t["amount"] for t in transactions if t["type"] == "income")
    expense_total = sum(t["amount"] for t in transactions if t["type"] == "expense")
    if income_total == 0:
        print("No income recorded. Please add income to get suggestions.")
        return
    expense_ratio = expense_total / income_total
    print("\n--- Budget Suggestions ---")
    if expense_ratio > 0.8:
        print("⚠️ You're spending more than 80% of your income. Cut back on discretionary items.")
    elif expense_ratio > 0.5:
        print("🙂 You're spending a moderate amount. Aim to save 20-30% of your income.")
    else:
        print("✅ Great job! You're saving a healthy portion of your income.")

# Simple menu-driven loop
def run_budget_advisor():
    load_data()
    while True:
        print("\n--- Menu ---")
        print("1. Add Transaction")
        print("2. View Summary")
        print("3. Analyze Spending")
        print("4. Budget Suggestions")
        print("5. Save & Exit")
        choice = input("Enter your choice (1-5): ").strip()
        if choice == '1':
            add_transaction()
        elif choice == '2':
            view_summary()
        elif choice == '3':
            analyze_spending()
        elif choice == '4':
            suggest_savings()
        elif choice == '5':
            save_data()
            print("Exiting... Have a financially wise day!")
            break
        else:
            print("Invalid choice. Please enter a number between 1 and 5.")

# Start the app
run_budget_advisor()


Data loaded successfully.

--- Menu ---
1. Add Transaction
2. View Summary
3. Analyze Spending
4. Budget Suggestions
5. Save & Exit
Transaction added successfully!

--- Menu ---
1. Add Transaction
2. View Summary
3. Analyze Spending
4. Budget Suggestions
5. Save & Exit

--- Summary ---
Total Income: ₹9914798.00
Total Expenses: ₹0.00
Balance: ₹9914798.00
Percentage of Income Spent: 0.00%

--- Menu ---
1. Add Transaction
2. View Summary
3. Analyze Spending
4. Budget Suggestions
5. Save & Exit

--- Summary ---
Total Income: ₹9914798.00
Total Expenses: ₹0.00
Balance: ₹9914798.00
Percentage of Income Spent: 0.00%

--- Menu ---
1. Add Transaction
2. View Summary
3. Analyze Spending
4. Budget Suggestions
5. Save & Exit
